# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [ ]:
cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'—', value='', regex=True)
cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York"], "Raleigh": ["Carolina"], "Nashville": ["Nashville"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona"]}

city_names_temp = cities[['Metropolitan area','NHL']].dropna()
city_names = city_names_temp['Metropolitan area']
city_names_list = city_names.tolist()
#nhl_df['W/L Ratio'] = nhl_df['W']/nhl_df['L']

nhl_2018_list = nhl_df.index[nhl_df['year']==2018].tolist()
first_nhl_2018_index    = nhl_2018_list[0]
last_nhl_2018_index     = nhl_2018_list[len(nhl_2018_list)-1]+1

nhl_2018_df = nhl_df.iloc[first_nhl_2018_index:last_nhl_2018_index, :]
nhl_2018_df = nhl_2018_df.replace(to_replace=r'\*', value='', regex=True)

nhl_2018_df['Numeric Check'] = nhl_2018_df['GP'].str.isnumeric()
for index in range(last_nhl_2018_index):
    if nhl_2018_df['Numeric Check'][index] == False:
        nhl_2018_df = nhl_2018_df.drop([index])

nhl_2018_df = nhl_2018_df.reset_index()
last_nhl_2018_index = len(nhl_2018_df['team'])

# nhl_2018_df['W'] = nhl_2018_df.W.astype(float)
# nhl_2018_df['L'] = nhl_2018_df.L.astype(float)
# nhl_2018_df['W/L Ratio'] = nhl_2018_df['W']/nhl_2018_df['L']
nhl_2018_df['W/L Ratio'] = nhl_2018_df.W.astype(float)/(nhl_2018_df.L.astype(float)+nhl_2018_df.W.astype(float))

nhl_2018_wl_df = nhl_2018_df[['team', 'W/L Ratio']]
cities_pop_df = cities[['Metropolitan area', 'Population', 'NHL']].dropna()
cities_pop_df.rename(columns={'Metropolitan area': 'City', 'NHL': 'team'}, inplace=True)

nhl_2018_wl_df.insert(2, "City", "")
for index in range(last_nhl_2018_index):
    for names in city_names_list:
        for city in cities_dict[names]:
            if city in nhl_2018_wl_df.loc[index, 'team']:
                tmp = nhl_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                nhl_2018_wl_df.loc[index, 'team'] = tmp
                nhl_2018_wl_df.loc[index, 'City'] = city

nhl_2018_wl_df['City'] = nhl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in cities_dict.items():
    for index in range(last_nhl_2018_index):
        if nhl_2018_wl_df.loc[index, 'City'] in values:
            nhl_2018_wl_df.loc[index, 'City'] = keys

merged_2018_wl_df = pd.merge(nhl_2018_wl_df, cities_pop_df, how='left', on='City')
merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
clean_pop_df = clean_pop_df.Population.astype(float)
population_by_region = clean_pop_df.values.tolist()

clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
win_loss_by_region_tmp = clean_wl_df.values.tolist()
win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]

corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    games = ['NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'—', value='', regex=True)
    cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

    cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York"], "Raleigh": ["Carolina"], "Nashville": ["Nashville"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona"]}

    city_names_temp = cities[['Metropolitan area','NHL']].dropna()
    city_names = city_names_temp['Metropolitan area']
    city_names_list = city_names.tolist()
    #nhl_df['W/L Ratio'] = nhl_df['W']/nhl_df['L']

    nhl_2018_list = nhl_df.index[nhl_df['year']==2018].tolist()
    first_nhl_2018_index    = nhl_2018_list[0]
    last_nhl_2018_index     = nhl_2018_list[len(nhl_2018_list)-1]+1

    nhl_2018_df = nhl_df.iloc[first_nhl_2018_index:last_nhl_2018_index, :]
    nhl_2018_df = nhl_2018_df.replace(to_replace=r'\*', value='', regex=True)

    nhl_2018_df['Numeric Check'] = nhl_2018_df['GP'].str.isnumeric()
    for index in range(last_nhl_2018_index):
        if nhl_2018_df['Numeric Check'][index] == False:
            nhl_2018_df = nhl_2018_df.drop([index])

    nhl_2018_df = nhl_2018_df.reset_index()
    last_nhl_2018_index = len(nhl_2018_df['team'])

    # nhl_2018_df['W'] = nhl_2018_df.W.astype(float)
    # nhl_2018_df['L'] = nhl_2018_df.L.astype(float)
    # nhl_2018_df['W/L Ratio'] = nhl_2018_df['W']/nhl_2018_df['L']
    nhl_2018_df['W/L Ratio'] = nhl_2018_df.W.astype(float)/(nhl_2018_df.L.astype(float)+nhl_2018_df.W.astype(float))

    nhl_2018_wl_df = nhl_2018_df[['team', 'W/L Ratio']]
    cities_pop_df = cities[['Metropolitan area', 'Population', 'NHL']].dropna()
    cities_pop_df.rename(columns={'Metropolitan area': 'City', 'NHL': 'team'}, inplace=True)

    nhl_2018_wl_df.insert(2, "City", "")
    for index in range(last_nhl_2018_index):
        for names in city_names_list:
            for city in cities_dict[names]:
                if city in nhl_2018_wl_df.loc[index, 'team']:
                    tmp = nhl_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    nhl_2018_wl_df.loc[index, 'team'] = tmp
                    nhl_2018_wl_df.loc[index, 'City'] = city

    nhl_2018_wl_df['City'] = nhl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in cities_dict.items():
        for index in range(last_nhl_2018_index):
            if nhl_2018_wl_df.loc[index, 'City'] in values:
                nhl_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_wl_df = pd.merge(nhl_2018_wl_df, cities_pop_df, how='left', on='City')
    merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
    clean_pop_df = clean_pop_df.Population.astype(float)
    population_by_region = clean_pop_df.values.tolist()

    clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
    win_loss_by_region_tmp = clean_wl_df.values.tolist()
    win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]
    #raise NotImplementedError()

    # population_by_region = [] # pass in metropolitan area population from cities
    # win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [ ]:
cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'—', value='', regex=True)
cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Raleigh": ["Carolina"], "Nashville": ["Nashville"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte"]}

city_names_temp = cities[['Metropolitan area','NBA']].dropna()
city_names = city_names_temp['Metropolitan area']
city_names_list = city_names.tolist()
#nba_df['W/L Ratio'] = nba_df['W']/nba_df['L']

nba_2018_list = nba_df.index[nba_df['year']==2018].tolist()
first_nba_2018_index    = nba_2018_list[0]
last_nba_2018_index     = nba_2018_list[len(nba_2018_list)-1]+1

nba_2018_df = nba_df.iloc[first_nba_2018_index:last_nba_2018_index, :]
nba_2018_df = nba_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
nba_2018_df = nba_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

# nba_2018_df['Numeric Check'] = nba_2018_df['GP'].str.isnumeric()
# for index in range(last_nba_2018_index):
#     if nba_2018_df['Numeric Check'][index] == False:
#         nba_2018_df = nba_2018_df.drop([index])

nba_2018_df = nba_2018_df.reset_index()
last_nba_2018_index = len(nba_2018_df['team'])

# nba_2018_df['W'] = nba_2018_df.W.astype(float)
# nba_2018_df['L'] = nba_2018_df.L.astype(float)
# nba_2018_df['W/L Ratio'] = nba_2018_df['W']/nba_2018_df['L']
nba_2018_df['W/L Ratio'] = nba_2018_df.W.astype(float)/(nba_2018_df.L.astype(float)+nba_2018_df.W.astype(float))

nba_2018_wl_df = nba_2018_df[['team', 'W/L Ratio']]
cities_pop_df = cities[['Metropolitan area', 'Population', 'NBA']].dropna()
cities_pop_df.rename(columns={'Metropolitan area': 'City', 'NBA': 'team'}, inplace=True)

nba_2018_wl_df.insert(2, "City", "")
for index in range(last_nba_2018_index):
    for names in city_names_list:
        for city in cities_dict[names]:
            if city in nba_2018_wl_df.loc[index, 'team']:
                tmp = nba_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                nba_2018_wl_df.loc[index, 'team'] = tmp
                nba_2018_wl_df.loc[index, 'City'] = city

nba_2018_wl_df['City'] = nba_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in cities_dict.items():
    for index in range(last_nba_2018_index):
        if nba_2018_wl_df.loc[index, 'City'] in values:
            nba_2018_wl_df.loc[index, 'City'] = keys

merged_2018_wl_df = pd.merge(nba_2018_wl_df, cities_pop_df, how='left', on='City')
merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
clean_pop_df = clean_pop_df.Population.astype(float)
population_by_region = clean_pop_df.values.tolist()

clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
win_loss_by_region_tmp = clean_wl_df.values.tolist()
win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]

corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'—', value='', regex=True)
    cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

    cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Raleigh": ["Carolina"], "Nashville": ["Nashville"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte"]}

    city_names_temp = cities[['Metropolitan area','NBA']].dropna()
    city_names = city_names_temp['Metropolitan area']
    city_names_list = city_names.tolist()
    #nba_df['W/L Ratio'] = nba_df['W']/nba_df['L']

    nba_2018_list = nba_df.index[nba_df['year']==2018].tolist()
    first_nba_2018_index    = nba_2018_list[0]
    last_nba_2018_index     = nba_2018_list[len(nba_2018_list)-1]+1

    nba_2018_df = nba_df.iloc[first_nba_2018_index:last_nba_2018_index, :]
    nba_2018_df = nba_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
    nba_2018_df = nba_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

    # nba_2018_df['Numeric Check'] = nba_2018_df['GP'].str.isnumeric()
    # for index in range(last_nba_2018_index):
    #     if nba_2018_df['Numeric Check'][index] == False:
    #         nba_2018_df = nba_2018_df.drop([index])

    nba_2018_df = nba_2018_df.reset_index()
    last_nba_2018_index = len(nba_2018_df['team'])

    # nba_2018_df['W'] = nba_2018_df.W.astype(float)
    # nba_2018_df['L'] = nba_2018_df.L.astype(float)
    # nba_2018_df['W/L Ratio'] = nba_2018_df['W']/nba_2018_df['L']
    nba_2018_df['W/L Ratio'] = nba_2018_df.W.astype(float)/(nba_2018_df.L.astype(float)+nba_2018_df.W.astype(float))

    nba_2018_wl_df = nba_2018_df[['team', 'W/L Ratio']]
    cities_pop_df = cities[['Metropolitan area', 'Population', 'NBA']].dropna()
    cities_pop_df.rename(columns={'Metropolitan area': 'City', 'NBA': 'team'}, inplace=True)

    nba_2018_wl_df.insert(2, "City", "")
    for index in range(last_nba_2018_index):
        for names in city_names_list:
            for city in cities_dict[names]:
                if city in nba_2018_wl_df.loc[index, 'team']:
                    tmp = nba_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    nba_2018_wl_df.loc[index, 'team'] = tmp
                    nba_2018_wl_df.loc[index, 'City'] = city

    nba_2018_wl_df['City'] = nba_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in cities_dict.items():
        for index in range(last_nba_2018_index):
            if nba_2018_wl_df.loc[index, 'City'] in values:
                nba_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_wl_df = pd.merge(nba_2018_wl_df, cities_pop_df, how='left', on='City')
    merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
    clean_pop_df = clean_pop_df.Population.astype(float)
    population_by_region = clean_pop_df.values.tolist()

    clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
    win_loss_by_region_tmp = clean_wl_df.values.tolist()
    win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [ ]:
cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'—', value='', regex=True)
cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Raleigh": ["Carolina"], "Nashville": ["Nashville"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"]}

city_names_temp = cities[['Metropolitan area','MLB']].dropna()
city_names = city_names_temp['Metropolitan area']
city_names_list = city_names.tolist()
#mlb_df['W/L Ratio'] = mlb_df['W']/mlb_df['L']

mlb_2018_list = mlb_df.index[mlb_df['year']==2018].tolist()
first_mlb_2018_index    = mlb_2018_list[0]
last_mlb_2018_index     = mlb_2018_list[len(mlb_2018_list)-1]+1

mlb_2018_df = mlb_df.iloc[first_mlb_2018_index:last_mlb_2018_index, :]
# mlb_2018_df = mlb_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
# mlb_2018_df = mlb_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

# mlb_2018_df['Numeric Check'] = mlb_2018_df['GP'].str.isnumeric()
# for index in range(last_mlb_2018_index):
#     if mlb_2018_df['Numeric Check'][index] == False:
#         mlb_2018_df = mlb_2018_df.drop([index])

mlb_2018_df = mlb_2018_df.reset_index()
last_mlb_2018_index = len(mlb_2018_df['team'])

# mlb_2018_df['W'] = mlb_2018_df.W.astype(float)
# mlb_2018_df['L'] = mlb_2018_df.L.astype(float)
# mlb_2018_df['W/L Ratio'] = mlb_2018_df['W']/mlb_2018_df['L']
mlb_2018_df['W/L Ratio'] = mlb_2018_df.W.astype(float)/(mlb_2018_df.L.astype(float)+mlb_2018_df.W.astype(float))

mlb_2018_wl_df = mlb_2018_df[['team', 'W/L Ratio']]
cities_pop_df = cities[['Metropolitan area', 'Population', 'MLB']].dropna()
cities_pop_df.rename(columns={'Metropolitan area': 'City', 'MLB': 'team'}, inplace=True)

mlb_2018_wl_df.insert(2, "City", "")
for index in range(last_mlb_2018_index):
    for names in city_names_list:
        for city in cities_dict[names]:
            if city in mlb_2018_wl_df.loc[index, 'team']:
                tmp = mlb_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                mlb_2018_wl_df.loc[index, 'team'] = tmp
                mlb_2018_wl_df.loc[index, 'City'] = city

mlb_2018_wl_df['City'] = mlb_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in cities_dict.items():
    for index in range(last_mlb_2018_index):
        if mlb_2018_wl_df.loc[index, 'City'] in values:
            mlb_2018_wl_df.loc[index, 'City'] = keys

merged_2018_wl_df = pd.merge(mlb_2018_wl_df, cities_pop_df, how='left', on='City')
merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
clean_pop_df = clean_pop_df.Population.astype(float)
population_by_region = clean_pop_df.values.tolist()

clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
win_loss_by_region_tmp = clean_wl_df.values.tolist()
win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]

corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'—', value='', regex=True)
    cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

    cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Raleigh": ["Carolina"], "Nashville": ["Nashville"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"]}

    city_names_temp = cities[['Metropolitan area','MLB']].dropna()
    city_names = city_names_temp['Metropolitan area']
    city_names_list = city_names.tolist()
    #mlb_df['W/L Ratio'] = mlb_df['W']/mlb_df['L']

    mlb_2018_list = mlb_df.index[mlb_df['year']==2018].tolist()
    first_mlb_2018_index    = mlb_2018_list[0]
    last_mlb_2018_index     = mlb_2018_list[len(mlb_2018_list)-1]+1

    mlb_2018_df = mlb_df.iloc[first_mlb_2018_index:last_mlb_2018_index, :]
    # mlb_2018_df = mlb_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
    # mlb_2018_df = mlb_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

    # mlb_2018_df['Numeric Check'] = mlb_2018_df['GP'].str.isnumeric()
    # for index in range(last_mlb_2018_index):
    #     if mlb_2018_df['Numeric Check'][index] == False:
    #         mlb_2018_df = mlb_2018_df.drop([index])

    mlb_2018_df = mlb_2018_df.reset_index()
    last_mlb_2018_index = len(mlb_2018_df['team'])

    # mlb_2018_df['W'] = mlb_2018_df.W.astype(float)
    # mlb_2018_df['L'] = mlb_2018_df.L.astype(float)
    # mlb_2018_df['W/L Ratio'] = mlb_2018_df['W']/mlb_2018_df['L']
    mlb_2018_df['W/L Ratio'] = mlb_2018_df.W.astype(float)/(mlb_2018_df.L.astype(float)+mlb_2018_df.W.astype(float))

    mlb_2018_wl_df = mlb_2018_df[['team', 'W/L Ratio']]
    cities_pop_df = cities[['Metropolitan area', 'Population', 'MLB']].dropna()
    cities_pop_df.rename(columns={'Metropolitan area': 'City', 'MLB': 'team'}, inplace=True)

    mlb_2018_wl_df.insert(2, "City", "")
    for index in range(last_mlb_2018_index):
        for names in city_names_list:
            for city in cities_dict[names]:
                if city in mlb_2018_wl_df.loc[index, 'team']:
                    tmp = mlb_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    mlb_2018_wl_df.loc[index, 'team'] = tmp
                    mlb_2018_wl_df.loc[index, 'City'] = city

    mlb_2018_wl_df['City'] = mlb_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in cities_dict.items():
        for index in range(last_mlb_2018_index):
            if mlb_2018_wl_df.loc[index, 'City'] in values:
                mlb_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_wl_df = pd.merge(mlb_2018_wl_df, cities_pop_df, how='left', on='City')
    merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
    clean_pop_df = clean_pop_df.Population.astype(float)
    population_by_region = clean_pop_df.values.tolist()

    clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
    win_loss_by_region_tmp = clean_wl_df.values.tolist()
    win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]
    #raise NotImplementedError()
    
    # population_by_region = [] # pass in metropolitan area population from cities
    # win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [ ]:
cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'—', value='', regex=True)
cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston", "New England"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Nashville": ["Nashville", "Tennessee"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte", "Carolina"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"], "Jacksonville": ["Jacksonville"], "Green Bay": ["Green Bay"]}

city_names_temp = cities[['Metropolitan area','NFL']].dropna()
city_names = city_names_temp['Metropolitan area']
city_names_list = city_names.tolist()
#nfl_df['W/L Ratio'] = nfl_df['W']/nfl_df['L']

nfl_2018_list = nfl_df.index[nfl_df['year']==2018].tolist()
first_nfl_2018_index    = nfl_2018_list[0]
last_nfl_2018_index     = nfl_2018_list[len(nfl_2018_list)-1]+1

nfl_2018_df = nfl_df.iloc[first_nfl_2018_index:last_nfl_2018_index, :]
nfl_2018_df = nfl_2018_df.replace(to_replace=r'\*', value='', regex=True)
nfl_2018_df = nfl_2018_df.replace(to_replace=r'\+', value='', regex=True)

nfl_2018_df['Numeric Check'] = nfl_2018_df['PA'].str.isnumeric()
for index in range(last_nfl_2018_index):
    if nfl_2018_df['Numeric Check'][index] == False:
        nfl_2018_df = nfl_2018_df.drop([index])

nfl_2018_df = nfl_2018_df.reset_index()
last_nfl_2018_index = len(nfl_2018_df['team'])

# nfl_2018_df['W'] = nfl_2018_df.W.astype(float)
# nfl_2018_df['L'] = nfl_2018_df.L.astype(float)
# nfl_2018_df['W/L Ratio'] = nfl_2018_df['W']/nfl_2018_df['L']
nfl_2018_df['W/L Ratio'] = nfl_2018_df.W.astype(float)/(nfl_2018_df.L.astype(float)+nfl_2018_df.W.astype(float))

nfl_2018_wl_df = nfl_2018_df[['team', 'W/L Ratio']]
cities_pop_df = cities[['Metropolitan area', 'Population', 'NFL']].dropna()
cities_pop_df.rename(columns={'Metropolitan area': 'City', 'NFL': 'team'}, inplace=True)

nfl_2018_wl_df.insert(2, "City", "")
for index in range(last_nfl_2018_index):
    for names in city_names_list:
        for city in cities_dict[names]:
            if city in nfl_2018_wl_df.loc[index, 'team']:
                tmp = nfl_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                nfl_2018_wl_df.loc[index, 'team'] = tmp
                nfl_2018_wl_df.loc[index, 'City'] = city

nfl_2018_wl_df['City'] = nfl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in cities_dict.items():
    for index in range(last_nfl_2018_index):
        if nfl_2018_wl_df.loc[index, 'City'] in values:
            nfl_2018_wl_df.loc[index, 'City'] = keys

merged_2018_wl_df = pd.merge(nfl_2018_wl_df, cities_pop_df, how='left', on='City')
merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
clean_pop_df = clean_pop_df.Population.astype(float)
population_by_region = clean_pop_df.values.tolist()

clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
win_loss_by_region_tmp = clean_wl_df.values.tolist()
win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]

corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'—', value='', regex=True)
    cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

    cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston", "New England"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Raleigh": ["Carolina"], "Nashville": ["Nashville", "Tennessee"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte", "Carolina"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"], "Jacksonville": ["Jacksonville"], "Green Bay": ["Green Bay"]}

    city_names_temp = cities[['Metropolitan area','NFL']].dropna()
    city_names = city_names_temp['Metropolitan area']
    city_names_list = city_names.tolist()
    #nfl_df['W/L Ratio'] = nfl_df['W']/nfl_df['L']

    nfl_2018_list = nfl_df.index[nfl_df['year']==2018].tolist()
    first_nfl_2018_index    = nfl_2018_list[0]
    last_nfl_2018_index     = nfl_2018_list[len(nfl_2018_list)-1]+1

    nfl_2018_df = nfl_df.iloc[first_nfl_2018_index:last_nfl_2018_index, :]
    nfl_2018_df = nfl_2018_df.replace(to_replace=r'\*', value='', regex=True)
    nfl_2018_df = nfl_2018_df.replace(to_replace=r'\+', value='', regex=True)

    nfl_2018_df['Numeric Check'] = nfl_2018_df['PA'].str.isnumeric()
    for index in range(last_nfl_2018_index):
        if nfl_2018_df['Numeric Check'][index] == False:
            nfl_2018_df = nfl_2018_df.drop([index])

    nfl_2018_df = nfl_2018_df.reset_index()
    last_nfl_2018_index = len(nfl_2018_df['team'])

    # nfl_2018_df['W'] = nfl_2018_df.W.astype(float)
    # nfl_2018_df['L'] = nfl_2018_df.L.astype(float)
    # nfl_2018_df['W/L Ratio'] = nfl_2018_df['W']/nfl_2018_df['L']
    nfl_2018_df['W/L Ratio'] = nfl_2018_df.W.astype(float)/(nfl_2018_df.L.astype(float)+nfl_2018_df.W.astype(float))

    nfl_2018_wl_df = nfl_2018_df[['team', 'W/L Ratio']]
    cities_pop_df = cities[['Metropolitan area', 'Population', 'NFL']].dropna()
    cities_pop_df.rename(columns={'Metropolitan area': 'City', 'NFL': 'team'}, inplace=True)

    nfl_2018_wl_df.insert(2, "City", "")
    for index in range(last_nfl_2018_index):
        for names in city_names_list:
            for city in cities_dict[names]:
                if city in nfl_2018_wl_df.loc[index, 'team']:
                    tmp = nfl_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    nfl_2018_wl_df.loc[index, 'team'] = tmp
                    nfl_2018_wl_df.loc[index, 'City'] = city

    nfl_2018_wl_df['City'] = nfl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in cities_dict.items():
        for index in range(last_nfl_2018_index):
            if nfl_2018_wl_df.loc[index, 'City'] in values:
                nfl_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_wl_df = pd.merge(nfl_2018_wl_df, cities_pop_df, how='left', on='City')
    merged_2018_wl_df = merged_2018_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_pop_df = merged_2018_wl_df.groupby(['City', 'Population'], axis=0).count().reset_index().drop(columns=["W/L Ratio"]).set_index("City")
    clean_pop_df = clean_pop_df.Population.astype(float)
    population_by_region = clean_pop_df.values.tolist()

    clean_wl_df = merged_2018_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")
    win_loss_by_region_tmp = clean_wl_df.values.tolist()
    win_loss_by_region = [item for sublist in win_loss_by_region_tmp for item in sublist]
    #raise NotImplementedError()
    
    # population_by_region = [] # pass in metropolitan area population from cities
    # win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [6]:
cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
cities = cities.replace(to_replace=r'—', value='', regex=True)
cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

nfl_cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston", "New England"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"],  "Nashville": ["Nashville", "Tennessee"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte", "Carolina"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"], "Jacksonville": ["Jacksonville"], "Green Bay": ["Green Bay"]}

cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston", "New England"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Raleigh": ["Carolina"], "Nashville": ["Nashville", "Tennessee"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte", "Carolina"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"], "Jacksonville": ["Jacksonville"], "Green Bay": ["Green Bay"]}

city_names_temp_nfl = cities[['Metropolitan area','NFL']].dropna()
city_names_nfl = city_names_temp_nfl['Metropolitan area']
city_names_nfl_list = city_names_nfl.tolist()
#nfl_df['W/L Ratio'] = nfl_df['W']/nfl_df['L']

nfl_2018_list = nfl_df.index[nfl_df['year']==2018].tolist()
first_nfl_2018_index    = nfl_2018_list[0]
last_nfl_2018_index     = nfl_2018_list[len(nfl_2018_list)-1]+1

nfl_2018_df = nfl_df.iloc[first_nfl_2018_index:last_nfl_2018_index, :]
nfl_2018_df = nfl_2018_df.replace(to_replace=r'\*', value='', regex=True)
nfl_2018_df = nfl_2018_df.replace(to_replace=r'\+', value='', regex=True)

nfl_2018_df['Numeric Check'] = nfl_2018_df['PA'].str.isnumeric()
for index in range(last_nfl_2018_index):
    if nfl_2018_df['Numeric Check'][index] == False:
        nfl_2018_df = nfl_2018_df.drop([index])

nfl_2018_df = nfl_2018_df.reset_index()
last_nfl_2018_index = len(nfl_2018_df['team'])

# nfl_2018_df['W'] = nfl_2018_df.W.astype(float)
# nfl_2018_df['L'] = nfl_2018_df.L.astype(float)
# nfl_2018_df['W/L Ratio'] = nfl_2018_df['W']/nfl_2018_df['L']
nfl_2018_df['W/L Ratio'] = nfl_2018_df.W.astype(float)/(nfl_2018_df.L.astype(float)+nfl_2018_df.W.astype(float))

nfl_2018_wl_df = nfl_2018_df[['team', 'W/L Ratio']]
cities_nfl_pop_df = cities[['Metropolitan area', 'Population', 'NFL']].dropna()
cities_nfl_pop_df.rename(columns={'Metropolitan area': 'City', 'NFL': 'team'}, inplace=True)

nfl_2018_wl_df.insert(2, "City", "")
for index in range(last_nfl_2018_index):
    for names in city_names_nfl_list:
        for city in nfl_cities_dict[names]:
            if city in nfl_2018_wl_df.loc[index, 'team']:
                tmp = nfl_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                nfl_2018_wl_df.loc[index, 'team'] = tmp
                nfl_2018_wl_df.loc[index, 'City'] = city

nfl_2018_wl_df['City'] = nfl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in nfl_cities_dict.items():
    for index in range(last_nfl_2018_index):
        if nfl_2018_wl_df.loc[index, 'City'] in values:
            nfl_2018_wl_df.loc[index, 'City'] = keys

merged_2018_nfl_wl_df = pd.merge(nfl_2018_wl_df, cities_nfl_pop_df, how='left', on='City')
merged_2018_nfl_wl_df = merged_2018_nfl_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_wl_nfl_df = merged_2018_nfl_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")


city_names_temp_mlb = cities[['Metropolitan area','MLB']].dropna()
city_names_mlb = city_names_temp_mlb['Metropolitan area']
city_names_mlb_list = city_names_mlb.tolist()

mlb_2018_list = mlb_df.index[mlb_df['year']==2018].tolist()
first_mlb_2018_index    = mlb_2018_list[0]
last_mlb_2018_index     = mlb_2018_list[len(mlb_2018_list)-1]+1

mlb_2018_df = mlb_df.iloc[first_mlb_2018_index:last_mlb_2018_index, :]
# mlb_2018_df = mlb_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
# mlb_2018_df = mlb_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

# mlb_2018_df['Numeric Check'] = mlb_2018_df['GP'].str.isnumeric()
# for index in range(last_mlb_2018_index):
#     if mlb_2018_df['Numeric Check'][index] == False:
#         mlb_2018_df = mlb_2018_df.drop([index])

mlb_2018_df = mlb_2018_df.reset_index()
last_mlb_2018_index = len(mlb_2018_df['team'])

# mlb_2018_df['W'] = mlb_2018_df.W.astype(float)
# mlb_2018_df['L'] = mlb_2018_df.L.astype(float)
# mlb_2018_df['W/L Ratio'] = mlb_2018_df['W']/mlb_2018_df['L']
mlb_2018_df['W/L Ratio'] = mlb_2018_df.W.astype(float)/(mlb_2018_df.L.astype(float)+mlb_2018_df.W.astype(float))

mlb_2018_wl_df = mlb_2018_df[['team', 'W/L Ratio']]
cities_mlb_pop_df = cities[['Metropolitan area', 'Population', 'MLB']].dropna()
cities_mlb_pop_df.rename(columns={'Metropolitan area': 'City', 'MLB': 'team'}, inplace=True)

mlb_2018_wl_df.insert(2, "City", "")
for index in range(last_mlb_2018_index):
    for names in city_names_mlb_list:
        for city in cities_dict[names]:
            if city in mlb_2018_wl_df.loc[index, 'team']:
                tmp = mlb_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                mlb_2018_wl_df.loc[index, 'team'] = tmp
                mlb_2018_wl_df.loc[index, 'City'] = city

mlb_2018_wl_df['City'] = mlb_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in cities_dict.items():
    for index in range(last_mlb_2018_index):
        if mlb_2018_wl_df.loc[index, 'City'] in values:
            mlb_2018_wl_df.loc[index, 'City'] = keys

merged_2018_mlb_wl_df = pd.merge(mlb_2018_wl_df, cities_mlb_pop_df, how='left', on='City')
merged_2018_mlb_wl_df = merged_2018_mlb_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_mlb_wl_df = merged_2018_mlb_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")


city_names_temp_nba = cities[['Metropolitan area','NBA']].dropna()
city_names_nba = city_names_temp_nba['Metropolitan area']
city_names_nba_list = city_names_nba.tolist()

nba_2018_list = nba_df.index[nba_df['year']==2018].tolist()
first_nba_2018_index    = nba_2018_list[0]
last_nba_2018_index     = nba_2018_list[len(nba_2018_list)-1]+1

nba_2018_df = nba_df.iloc[first_nba_2018_index:last_nba_2018_index, :]
nba_2018_df = nba_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
nba_2018_df = nba_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

# nba_2018_df['Numeric Check'] = nba_2018_df['GP'].str.isnumeric()
# for index in range(last_nba_2018_index):
#     if nba_2018_df['Numeric Check'][index] == False:
#         nba_2018_df = nba_2018_df.drop([index])

nba_2018_df = nba_2018_df.reset_index()
last_nba_2018_index = len(nba_2018_df['team'])

# nba_2018_df['W'] = nba_2018_df.W.astype(float)
# nba_2018_df['L'] = nba_2018_df.L.astype(float)
# nba_2018_df['W/L Ratio'] = nba_2018_df['W']/nba_2018_df['L']
nba_2018_df['W/L Ratio'] = nba_2018_df.W.astype(float)/(nba_2018_df.L.astype(float)+nba_2018_df.W.astype(float))

nba_2018_wl_df = nba_2018_df[['team', 'W/L Ratio']]
cities_nba_pop_df = cities[['Metropolitan area', 'Population', 'NBA']].dropna()
cities_nba_pop_df.rename(columns={'Metropolitan area': 'City', 'NBA': 'team'}, inplace=True)

nba_2018_wl_df.insert(2, "City", "")
for index in range(last_nba_2018_index):
    for names in city_names_nba_list:
        for city in cities_dict[names]:
            if city in nba_2018_wl_df.loc[index, 'team']:
                tmp = nba_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                nba_2018_wl_df.loc[index, 'team'] = tmp
                nba_2018_wl_df.loc[index, 'City'] = city

nba_2018_wl_df['City'] = nba_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in cities_dict.items():
    for index in range(last_nba_2018_index):
        if nba_2018_wl_df.loc[index, 'City'] in values:
            nba_2018_wl_df.loc[index, 'City'] = keys

merged_2018_nba_wl_df = pd.merge(nba_2018_wl_df, cities_nba_pop_df, how='left', on='City')
merged_2018_nba_wl_df = merged_2018_nba_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_nba_wl_df = merged_2018_nba_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")


city_names_temp_nhl = cities[['Metropolitan area','NHL']].dropna()
city_names_nhl = city_names_temp_nhl['Metropolitan area']
city_names_nhl_list = city_names_nhl.tolist()

nhl_2018_list = nhl_df.index[nhl_df['year']==2018].tolist()
first_nhl_2018_index    = nhl_2018_list[0]
last_nhl_2018_index     = nhl_2018_list[len(nhl_2018_list)-1]+1

nhl_2018_df = nhl_df.iloc[first_nhl_2018_index:last_nhl_2018_index, :]
nhl_2018_df = nhl_2018_df.replace(to_replace=r'\*', value='', regex=True)

nhl_2018_df['Numeric Check'] = nhl_2018_df['GP'].str.isnumeric()
for index in range(last_nhl_2018_index):
    if nhl_2018_df['Numeric Check'][index] == False:
        nhl_2018_df = nhl_2018_df.drop([index])

nhl_2018_df = nhl_2018_df.reset_index()
last_nhl_2018_index = len(nhl_2018_df['team'])

# nhl_2018_df['W'] = nhl_2018_df.W.astype(float)
# nhl_2018_df['L'] = nhl_2018_df.L.astype(float)
# nhl_2018_df['W/L Ratio'] = nhl_2018_df['W']/nhl_2018_df['L']
nhl_2018_df['W/L Ratio'] = nhl_2018_df.W.astype(float)/(nhl_2018_df.L.astype(float)+nhl_2018_df.W.astype(float))

nhl_2018_wl_df = nhl_2018_df[['team', 'W/L Ratio']]
cities_nhl_pop_df = cities[['Metropolitan area', 'Population', 'NHL']].dropna()
cities_nhl_pop_df.rename(columns={'Metropolitan area': 'City', 'NHL': 'team'}, inplace=True)

nhl_2018_wl_df.insert(2, "City", "")
for index in range(last_nhl_2018_index):
    for names in city_names_nhl_list:
        for city in cities_dict[names]:
            if city in nhl_2018_wl_df.loc[index, 'team']:
                tmp = nhl_2018_wl_df.loc[index, 'team']
                tmp = tmp.lstrip(city)
                nhl_2018_wl_df.loc[index, 'team'] = tmp
                nhl_2018_wl_df.loc[index, 'City'] = city

nhl_2018_wl_df['City'] = nhl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

for keys, values in cities_dict.items():
    for index in range(last_nhl_2018_index):
        if nhl_2018_wl_df.loc[index, 'City'] in values:
            nhl_2018_wl_df.loc[index, 'City'] = keys

merged_2018_nhl_wl_df = pd.merge(nhl_2018_wl_df, cities_nhl_pop_df, how='left', on='City')
merged_2018_nhl_wl_df = merged_2018_nhl_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

clean_nhl_wl_df = merged_2018_nhl_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")

nba_nfl_df = pd.merge(clean_nba_wl_df, clean_wl_nfl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NBA", "W/L Ratio_y": "NFL"}).dropna()
nba_nhl_df = pd.merge(clean_nba_wl_df, clean_nhl_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NBA", "W/L Ratio_y": "NHL"}).dropna()
nba_mlb_df = pd.merge(clean_nba_wl_df, clean_mlb_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NBA", "W/L Ratio_y": "MLB"}).dropna()
nfl_nhl_df = pd.merge(clean_wl_nfl_df, clean_nhl_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NFL", "W/L Ratio_y": "NHL"}).dropna()
nfl_mlb_df = pd.merge(clean_wl_nfl_df, clean_mlb_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NFL", "W/L Ratio_y": "MLB"}).dropna()
nhl_mlb_df = pd.merge(clean_nhl_wl_df, clean_mlb_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NHL", "W/L Ratio_y": "MLB"}).dropna()

teststat_nba_nfl, pval_nba_nfl = stats.ttest_rel(nba_nfl_df['NBA'], nba_nfl_df['NFL'])
teststat_nba_nhl, pval_nba_nhl = stats.ttest_rel(nba_nhl_df['NBA'], nba_nhl_df['NHL'])
teststat_nba_mlb, pval_nba_mlb = stats.ttest_rel(nba_mlb_df['NBA'], nba_mlb_df['MLB'])
teststat_nfl_nhl, pval_nfl_nhl = stats.ttest_rel(nfl_nhl_df['NFL'], nfl_nhl_df['NHL'])
teststat_nfl_mlb, pval_nfl_mlb = stats.ttest_rel(nfl_mlb_df['NFL'], nfl_mlb_df['MLB'])
teststat_nhl_mlb, pval_nhl_mlb = stats.ttest_rel(nhl_mlb_df['NHL'], nhl_mlb_df['MLB'])

teststat_nba_nba, pval_nba_nba = stats.ttest_rel(nba_nfl_df['NBA'], nba_nfl_df['NBA'])

pval_dict = {"Sport": ["NBA", "NFL", "NHL", "MLB"], 
            "NBA": [pval_nba_nba, pval_nba_nfl, pval_nba_nhl, pval_nba_mlb], 
            "NFL": [pval_nba_nfl, pval_nba_nba, pval_nfl_nhl, pval_nfl_mlb],
            "NHL": [pval_nba_nhl, pval_nfl_nhl, pval_nba_nba, pval_nhl_mlb],
            "MLB": [pval_nba_mlb, pval_nfl_mlb, pval_nhl_mlb, pval_nba_nba]}

p_values = pd.DataFrame(pval_dict).set_index("Sport")

len_dict = {"Sport": ["NBA", "NFL", "NHL", "MLB"], 
            "NBA": [len(clean_nba_wl_df), len(nba_nfl_df), len(nba_nhl_df), len(nba_mlb_df)], 
            "NFL": [len(nba_nfl_df), len(clean_wl_nfl_df), len(nfl_nhl_df), len(nfl_mlb_df)],
            "NHL": [len(nba_nhl_df), len(nfl_nhl_df), len(clean_nhl_wl_df), len(nhl_mlb_df)],
            "MLB": [len(nba_mlb_df), len(nfl_mlb_df), len(nhl_mlb_df), len(clean_mlb_wl_df)]}

len_df = pd.DataFrame(len_dict).set_index("Sport")

C:\Users\waele\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-6-baeb298197e8>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_2018_wl_df['City'] = nfl_2018_wl_df['City'].replace(to_replace='', value=np.nan)
C:\Users\waele\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\L

In [7]:
p_values
#assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"

,NBA,NFL,NHL,MLB
Sport,,,,
NBA,NaN,0.941792,0.022297,0.950540
NFL,0.941792,NaN,0.030883,0.802069
NHL,0.022297,0.030883,NaN,0.000708
MLB,0.950540,0.802069,0.000708,NaN


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities = cities.replace(to_replace=r' \[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'\[note [0-9].*\]', value='—', regex=True)
    cities = cities.replace(to_replace=r'—', value='', regex=True)
    cities = cities.replace(r'^\s*$', value=np.nan, regex=True)
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

    nfl_cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston", "New England"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Nashville": ["Nashville", "Tennessee"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte", "Carolina"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"], "Jacksonville": ["Jacksonville"], "Green Bay": ["Green Bay"]}
    
    cities_dict = {"Tampa Bay Area": ["Tampa Bay"], "Boston": ["Boston", "New England"], "Toronto": ["Toronto"], "Miami–Fort Lauderdale": ["Florida", "Miami"], "Detroit": ["Detroit"], "Montreal": ["Montreal"], "Ottawa": ["Ottawa"], "Buffalo": ["Buffalo"], "Washington, D.C.": ["Washington"], "Pittsburgh": ["Pittsburgh"], "Philadelphia": ["Philadelphia"], "Columbus": ["Columbus"], "New York City": ["New Jersey", "New York", "Brooklyn"], "Raleigh": ["Carolina"], "Nashville": ["Nashville", "Tennessee"], "Winnipeg": ["Winnipeg"], "Minneapolis–Saint Paul": ["Minnesota"], "Denver": ["Colorado", "Denver"], "St. Louis": ["St. Louis"], "Dallas–Fort Worth": ["Dallas", "Texas"], "Chicago": ["Chicago"], "Las Vegas": ["Vegas"], "Los Angeles": ["Anaheim", "Los Angeles"], "San Francisco Bay Area": ["San Jose", "Golden State", "San Francisco", "Oakland"], "Calgary": ["Calgary"], "Edmonton": ["Edmonton"], "Vancouver": ["Vancouver"], "Phoenix": ["Arizona", "Phoenix"], "Indianapolis": ["Indiana"], "Milwaukee": ["Milwaukee"], "Orlando": ["Orlando"], "Atlanta": ["Atlanta"], "Houston": ["Houston"], "Portland": ["Portland"], "Oklahoma City": ["Oklahoma City"], "Salt Lake City": ["Utah"], "New Orleans": ["New Orleans"], "San Antonio": ["San Antonio"], "Sacramento": ["Sacramento"], "Memphis": ["Memphis"], "Cleveland": ["Cleveland"], "Charlotte": ["Charlotte", "Carolina"], "Baltimore": ["Baltimore"], "Kansas City": ["Kansas City"], "San Diego": ["San Diego"], "Seattle": ["Seattle"], "Cincinnati": ["Cincinnati"], "Jacksonville": ["Jacksonville"], "Green Bay": ["Green Bay"]}

    city_names_temp_nfl = cities[['Metropolitan area','NFL']].dropna()
    city_names_nfl = city_names_temp_nfl['Metropolitan area']
    city_names_nfl_list = city_names_nfl.tolist()
    #nfl_df['W/L Ratio'] = nfl_df['W']/nfl_df['L']

    nfl_2018_list = nfl_df.index[nfl_df['year']==2018].tolist()
    first_nfl_2018_index    = nfl_2018_list[0]
    last_nfl_2018_index     = nfl_2018_list[len(nfl_2018_list)-1]+1

    nfl_2018_df = nfl_df.iloc[first_nfl_2018_index:last_nfl_2018_index, :]
    nfl_2018_df = nfl_2018_df.replace(to_replace=r'\*', value='', regex=True)
    nfl_2018_df = nfl_2018_df.replace(to_replace=r'\+', value='', regex=True)

    nfl_2018_df['Numeric Check'] = nfl_2018_df['PA'].str.isnumeric()
    for index in range(last_nfl_2018_index):
        if nfl_2018_df['Numeric Check'][index] == False:
            nfl_2018_df = nfl_2018_df.drop([index])

    nfl_2018_df = nfl_2018_df.reset_index()
    last_nfl_2018_index = len(nfl_2018_df['team'])

    # nfl_2018_df['W'] = nfl_2018_df.W.astype(float)
    # nfl_2018_df['L'] = nfl_2018_df.L.astype(float)
    # nfl_2018_df['W/L Ratio'] = nfl_2018_df['W']/nfl_2018_df['L']
    nfl_2018_df['W/L Ratio'] = nfl_2018_df.W.astype(float)/(nfl_2018_df.L.astype(float)+nfl_2018_df.W.astype(float))

    nfl_2018_wl_df = nfl_2018_df[['team', 'W/L Ratio']]
    cities_nfl_pop_df = cities[['Metropolitan area', 'Population', 'NFL']].dropna()
    cities_nfl_pop_df.rename(columns={'Metropolitan area': 'City', 'NFL': 'team'}, inplace=True)

    nfl_2018_wl_df.insert(2, "City", "")
    for index in range(last_nfl_2018_index):
        for names in city_names_nfl_list:
            for city in nfl_cities_dict[names]:
                if city in nfl_2018_wl_df.loc[index, 'team']:
                    tmp = nfl_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    nfl_2018_wl_df.loc[index, 'team'] = tmp
                    nfl_2018_wl_df.loc[index, 'City'] = city

    nfl_2018_wl_df['City'] = nfl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in nfl_cities_dict.items():
        for index in range(last_nfl_2018_index):
            if nfl_2018_wl_df.loc[index, 'City'] in values:
                nfl_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_nfl_wl_df = pd.merge(nfl_2018_wl_df, cities_nfl_pop_df, how='left', on='City')
    merged_2018_nfl_wl_df = merged_2018_nfl_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_wl_nfl_df = merged_2018_nfl_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")


    city_names_temp_mlb = cities[['Metropolitan area','MLB']].dropna()
    city_names_mlb = city_names_temp_mlb['Metropolitan area']
    city_names_mlb_list = city_names_mlb.tolist()

    mlb_2018_list = mlb_df.index[mlb_df['year']==2018].tolist()
    first_mlb_2018_index    = mlb_2018_list[0]
    last_mlb_2018_index     = mlb_2018_list[len(mlb_2018_list)-1]+1

    mlb_2018_df = mlb_df.iloc[first_mlb_2018_index:last_mlb_2018_index, :]
    # mlb_2018_df = mlb_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
    # mlb_2018_df = mlb_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

    # mlb_2018_df['Numeric Check'] = mlb_2018_df['GP'].str.isnumeric()
    # for index in range(last_mlb_2018_index):
    #     if mlb_2018_df['Numeric Check'][index] == False:
    #         mlb_2018_df = mlb_2018_df.drop([index])

    mlb_2018_df = mlb_2018_df.reset_index()
    last_mlb_2018_index = len(mlb_2018_df['team'])

    # mlb_2018_df['W'] = mlb_2018_df.W.astype(float)
    # mlb_2018_df['L'] = mlb_2018_df.L.astype(float)
    # mlb_2018_df['W/L Ratio'] = mlb_2018_df['W']/mlb_2018_df['L']
    mlb_2018_df['W/L Ratio'] = mlb_2018_df.W.astype(float)/(mlb_2018_df.L.astype(float)+mlb_2018_df.W.astype(float))

    mlb_2018_wl_df = mlb_2018_df[['team', 'W/L Ratio']]
    cities_mlb_pop_df = cities[['Metropolitan area', 'Population', 'MLB']].dropna()
    cities_mlb_pop_df.rename(columns={'Metropolitan area': 'City', 'MLB': 'team'}, inplace=True)

    mlb_2018_wl_df.insert(2, "City", "")
    for index in range(last_mlb_2018_index):
        for names in city_names_mlb_list:
            for city in cities_dict[names]:
                if city in mlb_2018_wl_df.loc[index, 'team']:
                    tmp = mlb_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    mlb_2018_wl_df.loc[index, 'team'] = tmp
                    mlb_2018_wl_df.loc[index, 'City'] = city

    mlb_2018_wl_df['City'] = mlb_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in cities_dict.items():
        for index in range(last_mlb_2018_index):
            if mlb_2018_wl_df.loc[index, 'City'] in values:
                mlb_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_mlb_wl_df = pd.merge(mlb_2018_wl_df, cities_mlb_pop_df, how='left', on='City')
    merged_2018_mlb_wl_df = merged_2018_mlb_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_mlb_wl_df = merged_2018_mlb_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")


    city_names_temp_nba = cities[['Metropolitan area','NBA']].dropna()
    city_names_nba = city_names_temp_nba['Metropolitan area']
    city_names_nba_list = city_names_nba.tolist()

    nba_2018_list = nba_df.index[nba_df['year']==2018].tolist()
    first_nba_2018_index    = nba_2018_list[0]
    last_nba_2018_index     = nba_2018_list[len(nba_2018_list)-1]+1

    nba_2018_df = nba_df.iloc[first_nba_2018_index:last_nba_2018_index, :]
    nba_2018_df = nba_2018_df.replace(to_replace=r'\* \(.*', value='', regex=True)
    nba_2018_df = nba_2018_df.replace(to_replace=r' \(.*', value='', regex=True)

    # nba_2018_df['Numeric Check'] = nba_2018_df['GP'].str.isnumeric()
    # for index in range(last_nba_2018_index):
    #     if nba_2018_df['Numeric Check'][index] == False:
    #         nba_2018_df = nba_2018_df.drop([index])

    nba_2018_df = nba_2018_df.reset_index()
    last_nba_2018_index = len(nba_2018_df['team'])

    # nba_2018_df['W'] = nba_2018_df.W.astype(float)
    # nba_2018_df['L'] = nba_2018_df.L.astype(float)
    # nba_2018_df['W/L Ratio'] = nba_2018_df['W']/nba_2018_df['L']
    nba_2018_df['W/L Ratio'] = nba_2018_df.W.astype(float)/(nba_2018_df.L.astype(float)+nba_2018_df.W.astype(float))

    nba_2018_wl_df = nba_2018_df[['team', 'W/L Ratio']]
    cities_nba_pop_df = cities[['Metropolitan area', 'Population', 'NBA']].dropna()
    cities_nba_pop_df.rename(columns={'Metropolitan area': 'City', 'NBA': 'team'}, inplace=True)

    nba_2018_wl_df.insert(2, "City", "")
    for index in range(last_nba_2018_index):
        for names in city_names_nba_list:
            for city in cities_dict[names]:
                if city in nba_2018_wl_df.loc[index, 'team']:
                    tmp = nba_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    nba_2018_wl_df.loc[index, 'team'] = tmp
                    nba_2018_wl_df.loc[index, 'City'] = city

    nba_2018_wl_df['City'] = nba_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in cities_dict.items():
        for index in range(last_nba_2018_index):
            if nba_2018_wl_df.loc[index, 'City'] in values:
                nba_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_nba_wl_df = pd.merge(nba_2018_wl_df, cities_nba_pop_df, how='left', on='City')
    merged_2018_nba_wl_df = merged_2018_nba_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_nba_wl_df = merged_2018_nba_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")


    city_names_temp_nhl = cities[['Metropolitan area','NHL']].dropna()
    city_names_nhl = city_names_temp_nhl['Metropolitan area']
    city_names_nhl_list = city_names_nhl.tolist()

    nhl_2018_list = nhl_df.index[nhl_df['year']==2018].tolist()
    first_nhl_2018_index    = nhl_2018_list[0]
    last_nhl_2018_index     = nhl_2018_list[len(nhl_2018_list)-1]+1

    nhl_2018_df = nhl_df.iloc[first_nhl_2018_index:last_nhl_2018_index, :]
    nhl_2018_df = nhl_2018_df.replace(to_replace=r'\*', value='', regex=True)

    nhl_2018_df['Numeric Check'] = nhl_2018_df['GP'].str.isnumeric()
    for index in range(last_nhl_2018_index):
        if nhl_2018_df['Numeric Check'][index] == False:
            nhl_2018_df = nhl_2018_df.drop([index])

    nhl_2018_df = nhl_2018_df.reset_index()
    last_nhl_2018_index = len(nhl_2018_df['team'])

    # nhl_2018_df['W'] = nhl_2018_df.W.astype(float)
    # nhl_2018_df['L'] = nhl_2018_df.L.astype(float)
    # nhl_2018_df['W/L Ratio'] = nhl_2018_df['W']/nhl_2018_df['L']
    nhl_2018_df['W/L Ratio'] = nhl_2018_df.W.astype(float)/(nhl_2018_df.L.astype(float)+nhl_2018_df.W.astype(float))

    nhl_2018_wl_df = nhl_2018_df[['team', 'W/L Ratio']]
    cities_nhl_pop_df = cities[['Metropolitan area', 'Population', 'NHL']].dropna()
    cities_nhl_pop_df.rename(columns={'Metropolitan area': 'City', 'NHL': 'team'}, inplace=True)

    nhl_2018_wl_df.insert(2, "City", "")
    for index in range(last_nhl_2018_index):
        for names in city_names_nhl_list:
            for city in cities_dict[names]:
                if city in nhl_2018_wl_df.loc[index, 'team']:
                    tmp = nhl_2018_wl_df.loc[index, 'team']
                    tmp = tmp.lstrip(city)
                    nhl_2018_wl_df.loc[index, 'team'] = tmp
                    nhl_2018_wl_df.loc[index, 'City'] = city

    nhl_2018_wl_df['City'] = nhl_2018_wl_df['City'].replace(to_replace='', value=np.nan)

    for keys, values in cities_dict.items():
        for index in range(last_nhl_2018_index):
            if nhl_2018_wl_df.loc[index, 'City'] in values:
                nhl_2018_wl_df.loc[index, 'City'] = keys

    merged_2018_nhl_wl_df = pd.merge(nhl_2018_wl_df, cities_nhl_pop_df, how='left', on='City')
    merged_2018_nhl_wl_df = merged_2018_nhl_wl_df.drop(columns=["team_x", "team_y"]).set_index("City")

    clean_nhl_wl_df = merged_2018_nhl_wl_df.groupby(['City'], axis=0).agg({'W/L Ratio': np.nanmean}).reset_index().set_index("City")


    nba_nfl_df = pd.merge(clean_nba_wl_df, clean_wl_nfl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NBA", "W/L Ratio_y": "NFL"}).dropna()
    nba_nhl_df = pd.merge(clean_nba_wl_df, clean_nhl_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NBA", "W/L Ratio_y": "NHL"}).dropna()
    nba_mlb_df = pd.merge(clean_nba_wl_df, clean_mlb_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NBA", "W/L Ratio_y": "MLB"}).dropna()
    nfl_nhl_df = pd.merge(clean_wl_nfl_df, clean_nhl_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NFL", "W/L Ratio_y": "NHL"}).dropna()
    nfl_mlb_df = pd.merge(clean_wl_nfl_df, clean_mlb_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NFL", "W/L Ratio_y": "MLB"}).dropna()
    nhl_mlb_df = pd.merge(clean_nhl_wl_df, clean_mlb_wl_df, how='left', on='City').rename(columns={"W/L Ratio_x": "NHL", "W/L Ratio_y": "MLB"}).dropna()

    teststat_nba_nfl, pval_nba_nfl = stats.ttest_rel(nba_nfl_df['NBA'], nba_nfl_df['NFL'])
    teststat_nba_nhl, pval_nba_nhl = stats.ttest_rel(nba_nhl_df['NBA'], nba_nhl_df['NHL'])
    teststat_nba_mlb, pval_nba_mlb = stats.ttest_rel(nba_mlb_df['NBA'], nba_mlb_df['MLB'])
    teststat_nfl_nhl, pval_nfl_nhl = stats.ttest_rel(nfl_nhl_df['NFL'], nfl_nhl_df['NHL'])
    teststat_nfl_mlb, pval_nfl_mlb = stats.ttest_rel(nfl_mlb_df['NFL'], nfl_mlb_df['MLB'])
    teststat_nhl_mlb, pval_nhl_mlb = stats.ttest_rel(nhl_mlb_df['NHL'], nhl_mlb_df['MLB'])

    teststat_nba_nba, pval_nba_nba = stats.ttest_rel(nba_nfl_df['NBA'], nba_nfl_df['NBA'])

    pval_dict = {"Sport": ["NBA", "NFL", "NHL", "MLB"], 
                "NBA": [pval_nba_nba, pval_nba_nfl, pval_nba_nhl, pval_nba_mlb], 
                "NFL": [pval_nba_nfl, pval_nba_nba, pval_nfl_nhl, pval_nfl_mlb],
                "NHL": [pval_nba_nhl, pval_nfl_nhl, pval_nba_nba, pval_nhl_mlb],
                "MLB": [pval_nba_mlb, pval_nfl_mlb, pval_nhl_mlb, pval_nba_nba]}

    p_values = pd.DataFrame(pval_dict).set_index("Sport")
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    # sports = ['NFL', 'NBA', 'NHL', 'MLB']
    # p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    print(p_values)
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values